In [ ]:
import rasterio
import rasterstats



#Open the .tif which contains the NDVI calculation with rasterio
ndvipath="Data_files/clipped_ndvi.tif"
ndvix=rasterio.open(ndvipath)
    
#Make our projected overlay vector the same CRS as the imagery
temp = merged.to_crs(crs=ndvix.crs.data)
    
#Define the parameters to calculate the zonal stats
array = ndvix.read(1)
affine = ndvix.transform
zs_ndvix = zonal_stats(temp, array, affine=affine, stats=['mean', 'min', 'max', 'std'])
    
#Append the results to a pandas' dataframe
ndviframe = pd.DataFrame(zs_ndvix)
resultado = resultado.join(ndviframe, rsuffix='_'+str(i))

resultado